In [ ]:
from transformers import pipeline
import pandas as pd
import os

print("--- TAREA EXTRA: ZERO-SHOT CLASSIFICATION CON PARADETOX ---")
print("Objetivo: Detectar toxicidad usando un CSV específico.\n")

# 1. Configuración del archivo y columna
archivo_csv = "datos/english_paradetox_test_set_preprocessed_non_lemmatized.csv"
columna_texto = "text"  # La columna que solicitaste

# 2. Cargamos el pipeline
print("Cargando modelo (esto puede tardar un poco)...")
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# 3. Etiquetas candidatas
candidate_labels = ["insult", "threat", "hate speech", "constructive criticism", "toxic"]

# 4. Carga y Procesamiento del CSV
if os.path.exists(archivo_csv):
    try:
        # Leemos el CSV
        df = pd.read_csv(archivo_csv)

        if columna_texto not in df.columns:
            raise ValueError(f"La columna '{columna_texto}' no existe en el CSV. Las columnas encontradas son: {list(df.columns)}")

        # IMPORTANTE: Tomamos solo las primeras 5 filas para la prueba.
        subset_df = df.head(5)

        print(f"\nProcesando las primeras {len(subset_df)} filas del archivo '{archivo_csv}'...\n")

        for index, row in subset_df.iterrows():
            texto_actual = row[columna_texto]

            # Verificación básica de que hay texto
            if pd.isna(texto_actual) or str(texto_actual).strip() == "":
                print(f"Fila {index}: Texto vacío, saltando...")
                continue

            # Clasificación
            result = classifier(str(texto_actual), candidate_labels)

            # Resultados
            top_label = result['labels'][0]
            top_score = result['scores'][0]

            print(f"Fila {index}:")
            print(f"Texto: '{texto_actual}'")
            print(f"Ganador: {top_label} (Confianza: {top_score:.2%})")
            print("-" * 30)

    except Exception as e:
        print(f"Ocurrió un error al procesar el CSV: {e}")
else:
    print(f"ERROR: No encuentro el archivo '{archivo_csv}' en esta carpeta.")
    print("Asegúrate de subir el archivo al mismo directorio donde corre este script.")

--- TAREA EXTRA: ZERO-SHOT CLASSIFICATION CON PARADETOX ---
Objetivo: Detectar toxicidad usando un CSV específico.

Cargando modelo (esto puede tardar un poco)...


Device set to use cuda:0



Procesando las primeras 5 filas del archivo 'datos/english_paradetox_test_set_preprocessed_non_lemmatized.csv'...

Fila 0:
Texto: 'mandated  and " right fucking now " would be good .'
Ganador: threat (Confianza: 89.00%)
------------------------------
Fila 1:
Texto: '&gt i wouldn 't care how ignorant you are in you weren 't pretending to know shit .'
Ganador: threat (Confianza: 49.58%)
------------------------------
Fila 2:
Texto: '6. are you fucking insane ?'
Ganador: insult (Confianza: 33.75%)
------------------------------
Fila 3:
Texto: 'after all its hard to get a job if your stupid .'
Ganador: threat (Confianza: 43.37%)
------------------------------
Fila 4:
Texto: 'after all most of these dorks are millionares and can pay their own way .'
Ganador: threat (Confianza: 47.58%)
------------------------------


In [ ]:
from transformers import pipeline
import pandas as pd
import os

print("--- CLASIFICACIÓN DE EMOCIONES (EJEMPLOS ALEATORIOS) ---")

# 1. Configuración
archivo_csv = "datos/english_paradetox_test_set_preprocessed_non_lemmatized.csv"
columna_texto = "text"
cantidad_ejemplos = 5 

# 2. Cargar pipeline de emociones
print("Cargando modelo de emociones...")
classifier_emotion = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

# 3. Cargar y procesar datos
if os.path.exists(archivo_csv):
    try:
        df = pd.read_csv(archivo_csv)

        # Verificar columna
        if columna_texto not in df.columns:
            raise ValueError(f"No existe la columna '{columna_texto}'")

        n_muestras = min(cantidad_ejemplos, len(df))
        subset_df = df.sample(n=n_muestras)

        print(f"\nAnalizando {n_muestras} frases ALEATORIAS del dataset...\n")

        for index, row in subset_df.iterrows():
            texto = row[columna_texto]

            # Saltamos vacíos
            if pd.isna(texto) or str(texto).strip() == "":
                continue

            # Clasificar
            resultado = classifier_emotion(str(texto))

            # Obtener la emoción ganadora (la primera de la lista ordenada)
            emocion = resultado[0][0]['label']
            confianza = resultado[0][0]['score']

            print(f"Fila {index}:")
            print(f"Texto: '{texto}'")
            print(f"Emoción predominante: {emocion.upper()} (Confianza: {confianza:.2f})")
            print("-" * 30)

    except Exception as e:
        print(f"Error: {e}")
else:
    print(f"No se encuentra el archivo: {archivo_csv}")

--- CLASIFICACIÓN DE EMOCIONES (EJEMPLOS ALEATORIOS) ---
Cargando modelo de emociones...


Device set to use cuda:0



Analizando 5 frases ALEATORIAS del dataset...

Fila 569:
Texto: 'Dont really like the liberals either, Too much lefty rubbish leaves us all poor.'
Emoción predominante: SADNESS (Confianza: 0.44)
------------------------------
Fila 62:
Texto: 'dont you ever change one of my edits , or i will come to your house , and i will cut you .'
Emoción predominante: ANGER (Confianza: 0.77)
------------------------------
Fila 253:
Texto: 'shit , i d love an number pm sunset .'
Emoción predominante: DISGUST (Confianza: 0.25)
------------------------------
Fila 67:
Texto: 'either way , how the fuck do you forget your child in the backseat .'
Emoción predominante: ANGER (Confianza: 0.81)
------------------------------
Fila 281:
Texto: 'that s a lot of work for something that shouldn t take any fucking work at all .'
Emoción predominante: ANGER (Confianza: 0.79)
------------------------------


In [ ]:
from transformers import pipeline

print("--- ANÁLISIS DE OBJETIVOS (NER) ---")
print("Objetivo: Identificar a QUÉ o QUIÉN se ataca (Personas, Organizaciones, Países).\n")

# 1. Cargar el pipeline de NER (Named Entity Recognition)
# aggregation_strategy="simple" es CLAVE: junta los tokens ("J", "##ohn") en palabras completas ("John")
ner_classifier = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

# 2. Ejemplos con entidades claras
ejemplos_ner = [
    "I hate John because he is stupid",              # Ataque a Persona
    "Google is the worst company ever",              # Ataque a Organización
    "I will destroy Paris and all its people",       # Ataque a Lugar
    "Americans are always so loud",                  # Ataque a Grupo (Gentilicio)
    "Kill Elon Musk right now"                       # Amenaza directa a Persona
]

# 3. Procesar y mostrar resultados
for texto in ejemplos_ner:
    entidades = ner_classifier(texto)

    print(f"Frase: '{texto}'")
    if len(entidades) > 0:
        for ent in entidades:
            print(f"  └── Entidad detectada: {ent['word']} (Tipo: {ent['entity_group']}, Confianza: {ent['score']:.2f})")
    else:
        print("  └── No se detectaron nombres propios específicos.")
    print("-" * 30)

c:\Users\js834\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--- ANÁLISIS DE OBJETIVOS (NER) ---
Objetivo: Identificar a QUÉ o QUIÉN se ataca (Personas, Organizaciones, Países).




Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


Frase: 'I hate John because he is stupid'
  └── Entidad detectada: John (Tipo: PER, Confianza: 1.00)
------------------------------
Frase: 'Google is the worst company ever'
  └── Entidad detectada: Google (Tipo: ORG, Confianza: 1.00)
------------------------------
Frase: 'I will destroy Paris and all its people'
  └── Entidad detectada: Paris (Tipo: LOC, Confianza: 1.00)
------------------------------
Frase: 'Americans are always so loud'
  └── Entidad detectada: Americans (Tipo: MISC, Confianza: 1.00)
------------------------------
Frase: 'Kill Elon Musk right now'
  └── Entidad detectada: El (Tipo: PER, Confianza: 0.92)
  └── Entidad detectada: ##on Musk (Tipo: PER, Confianza: 0.86)
------------------------------
